In [1]:
from os import listdir
from os.path import join, isfile, abspath, dirname

import sys
sys.path.insert(0, abspath(join(dirname("."), '..')))

from pathlib import Path

from modules.helpers.utils import path_decompose, get_git_root, get_seed_for_graph, get_graph_info_from_seed
from modules.helpers.runners import run_solver_against_configs
from modules.helpers.constants import PERCENTAGES, IMMUNIZATION_PERCENTAGES

In [2]:
DS_PATH = path_decompose("H:\\Research\\NIGraphs\\withPasive")
SEED_PATH = path_decompose("H:\\Research\\NIGraphs\\seeds")

full_config = ['Random', 'Degree', 'SparseShieldPlus',
               'SparseShieldSeedless', 'Dom', 'MeshShield']
experiment_config = ['MeshShield', 'Dom']
NUM_THREADS = 4 * len(experiment_config)

folder_root = get_git_root(".")
folder_for_results = join(path_decompose(folder_root), "results")

sys.path.append(folder_root)

Path(folder_for_results).mkdir(parents=True, exist_ok=True)

In [3]:
all_graphs = [f for f in listdir(DS_PATH) if isfile(join(DS_PATH, f))]

In [4]:
def run_to(graph, seed, startN, endN, stepN):
    run_solver_against_configs(results_path=folder_for_results,
                               graph_file=graph,
                               seed_file=seed,
                               startNumber=startN,
                               endNumber=endN, 
                               step=stepN, 
                               algorithms_to_run=experiment_config, 
                               just_solve=True,
                               num_threads=NUM_THREADS)

In [5]:
standard_budgets = [1, 5, 10]

for graph in all_graphs:
    graph_path = join(DS_PATH, graph)
    graph_name = graph.replace(".pkl", "")
    
    for percentage in PERCENTAGES:
        seed_name = get_seed_for_graph(graph_name, SEED_PATH, percentage)
        (num_nodes, num_edges, percentage) = get_graph_info_from_seed(seed_name)
        
        for n in standard_budgets:
            run_to(graph_path, seed_name, n, n+1, 1)

        for p in IMMUNIZATION_PERCENTAGES:
            budget = int(float(p)*float(num_nodes) + 1.0)
            run_to(graph_path, seed_name, budget, budget+1, 1)